In [7]:
"""
Führt Regressionsanalyse durch

"""
import pandas as pd
import numpy as np



#Initmessage
print ("starting Regression") 

#lade von CSV
data_frame_investigated = pd.read_csv("Data/crypto-markets.csv", sep="," )


print("data collected")

print(data_frame_investigated.head())

starting Regression
data collected
      slug symbol     name        date  ranknow    open    high     low  \
0  bitcoin    BTC  Bitcoin  2013-04-28        1  135.30  135.98  132.10   
1  bitcoin    BTC  Bitcoin  2013-04-29        1  134.44  147.49  134.00   
2  bitcoin    BTC  Bitcoin  2013-04-30        1  144.00  146.93  134.05   
3  bitcoin    BTC  Bitcoin  2013-05-01        1  139.00  139.89  107.72   
4  bitcoin    BTC  Bitcoin  2013-05-02        1  116.38  125.60   92.28   

    close  volume        market  close_ratio  spread  
0  134.21     0.0  1.500520e+09       0.5438    3.88  
1  144.54     0.0  1.491160e+09       0.7813   13.49  
2  139.00     0.0  1.597780e+09       0.3843   12.88  
3  116.99     0.0  1.542820e+09       0.2882   32.17  
4  105.21     0.0  1.292190e+09       0.3881   33.32  


In [8]:
#eliminiere redundante Attribute
REDUNDANT_ATTRIBUTES=['symbol','name'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True) #loeschen aus Dataframe

#set Date as Index


print(data_frame_investigated.head())


      slug        date  ranknow    open    high     low   close  volume  \
0  bitcoin  2013-04-28        1  135.30  135.98  132.10  134.21     0.0   
1  bitcoin  2013-04-29        1  134.44  147.49  134.00  144.54     0.0   
2  bitcoin  2013-04-30        1  144.00  146.93  134.05  139.00     0.0   
3  bitcoin  2013-05-01        1  139.00  139.89  107.72  116.99     0.0   
4  bitcoin  2013-05-02        1  116.38  125.60   92.28  105.21     0.0   

         market  close_ratio  spread  
0  1.500520e+09       0.5438    3.88  
1  1.491160e+09       0.7813   13.49  
2  1.597780e+09       0.3843   12.88  
3  1.542820e+09       0.2882   32.17  
4  1.292190e+09       0.3881   33.32  


In [9]:

from sklearn.model_selection import train_test_split #library fuer Split
from sklearn import linear_model #lineare Regression
import math #import fuer msqError
from sklearn.metrics import mean_squared_error

#dependency :patsy

def cryptoregress(crypto_id, feature_to_compare,crypto_id_feature_name="slug",
                 documentMode=False):
    """
    lineare Regression zur Bestimmung eines Features anhand gleichen Features der anderen Cryptos
    """
    #Cryptos Gruppieren
    
    #ich kann nur anhand von Tagen schliessen, an denen die Crypto Existiert
    #erstmal alle Tage holen an denen die Crypto gehandelt wird
    dates_of_crypto_trade= data_frame_investigated[
        data_frame_investigated[crypto_id_feature_name] == crypto_id
    ]['date']
    

   
    if documentMode: print(dates_of_crypto_trade.head())
    
    #dann die Tupel Filtern, nur die Tage an denen Crypto getraded wird uebernehmen,
    #nur compare feature uebernehmen
    data_frame_id_exists= data_frame_investigated[
        data_frame_investigated['date'].isin(dates_of_crypto_trade)
    ][['date',crypto_id_feature_name,feature_to_compare]]
    
    
    
    #daten Pivotieren so dass Cryptos zu columns werden
    data_frame_id_exists= data_frame_id_exists.pivot(
        index='date', columns=crypto_id_feature_name, values=feature_to_compare
        ).reset_index()
    
    
    #collumns loeschen die Nullwerte enthalten
    data_frame_id_exists=data_frame_id_exists.dropna(axis='columns')
 
    
   # data_frame_id_exists=data_frame_id_exists.fillna(value=0)
    
    
   
    #date rausloeschen
    data_frame_id_exists.drop('date',axis=1,inplace=True)
    if documentMode: print("cleaned Frame")
    if documentMode: print(data_frame_id_exists.head())
    
    ###############Train Test Split:
    #Ergebnis vom Features trennen:
    Y = data_frame_id_exists[[crypto_id]]
    data_frame_id_exists.drop(crypto_id,axis=1, inplace = True)
    if documentMode: print("Regress to:")
    if documentMode: print(Y.head())

    
        
    feature_train, feature_test, y_train, y_test = train_test_split (
        data_frame_id_exists,
        Y,
        test_size=0.1
    )

    ##############Regressionsmodell machen
    regression_model = linear_model.LinearRegression(
        fit_intercept=True, normalize=False, copy_X=True, n_jobs=-1)


        #regressionsmodell fitten
    regression_model.fit(feature_train, y_train)
    Rsquared_train = regression_model.score(feature_train,y_train)
    Rsqared_test = regression_model.score(feature_test,y_test)
    MeanError_train = math.sqrt( mean_squared_error(regression_model.predict(feature_test),y_test))
    MeanError_test = math.sqrt( mean_squared_error(regression_model.predict(feature_train),y_train))
                                    
    if documentMode: print("RSquared train:",Rsquared_train)
    if documentMode: print("RSquared test:",Rsqared_test)
    if documentMode: print("MeanSqure train:", MeanError_train)
    if documentMode: print("MeanSqure test:", MeanError_test)
    
    return [regression_model,
            Rsquared_train,
            Rsqared_test,
            MeanError_train,
            MeanError_test,
            list(data_frame_id_exists),
           ]
       

    


regression_result=cryptoregress('bitcoin','close')

    
    
    
    
    

In [10]:
print(regression_result)

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False), 0.9255020861856469, 0.8586151600286503, 990.2198872998928, 867.6250229059269, ['litecoin', 'namecoin']]


In [11]:
#make a  list of the different coins
different_coins = data_frame_investigated.slug.unique()
print(different_coins)

['bitcoin' 'ethereum' 'ripple' ... 'avoncoin' 'infchain' 'monero-gold']


In [13]:
#iteriere durch die Liste und fuehre Regressionsanalyse durch
list_regression = []
zaehler=0
for current_coin_to_regress in different_coins:
    list_regression.append(
        {
            'name':current_coin_to_regress,
            'RsquaredTest': cryptoregress(current_coin_to_regress,'close')[2]
        }) #fuege ergebnis zur Ergebnistabelle hinzu
    zaehler=zaehler+1
    print("regressed Number ", zaehler, " von ", len(different_coins))
    if zaehler == 1300: break
    
zaehler= 0
df_regression_result=pd.DataFrame(list_regression)
print(df_regression_result)

regressed Number  1  von  1586
regressed Number  2  von  1586
regressed Number  3  von  1586
regressed Number  4  von  1586
regressed Number  5  von  1586
regressed Number  6  von  1586
regressed Number  7  von  1586
regressed Number  8  von  1586
regressed Number  9  von  1586
regressed Number  10  von  1586
regressed Number  11  von  1586
regressed Number  12  von  1586
regressed Number  13  von  1586
regressed Number  14  von  1586
regressed Number  15  von  1586
regressed Number  16  von  1586
regressed Number  17  von  1586
regressed Number  18  von  1586
regressed Number  19  von  1586
regressed Number  20  von  1586
regressed Number  21  von  1586
regressed Number  22  von  1586
regressed Number  23  von  1586
regressed Number  24  von  1586
regressed Number  25  von  1586
regressed Number  26  von  1586
regressed Number  27  von  1586
regressed Number  28  von  1586
regressed Number  29  von  1586
regressed Number  30  von  1586
regressed Number  31  von  1586
regressed Number 

regressed Number  253  von  1586
regressed Number  254  von  1586
regressed Number  255  von  1586
regressed Number  256  von  1586
regressed Number  257  von  1586
regressed Number  258  von  1586
regressed Number  259  von  1586
regressed Number  260  von  1586
regressed Number  261  von  1586
regressed Number  262  von  1586
regressed Number  263  von  1586
regressed Number  264  von  1586
regressed Number  265  von  1586
regressed Number  266  von  1586
regressed Number  267  von  1586
regressed Number  268  von  1586
regressed Number  269  von  1586
regressed Number  270  von  1586
regressed Number  271  von  1586
regressed Number  272  von  1586
regressed Number  273  von  1586
regressed Number  274  von  1586
regressed Number  275  von  1586
regressed Number  276  von  1586
regressed Number  277  von  1586
regressed Number  278  von  1586
regressed Number  279  von  1586
regressed Number  280  von  1586
regressed Number  281  von  1586
regressed Number  282  von  1586
regressed 

regressed Number  502  von  1586
regressed Number  503  von  1586
regressed Number  504  von  1586
regressed Number  505  von  1586
regressed Number  506  von  1586
regressed Number  507  von  1586
regressed Number  508  von  1586
regressed Number  509  von  1586
regressed Number  510  von  1586
regressed Number  511  von  1586
regressed Number  512  von  1586
regressed Number  513  von  1586
regressed Number  514  von  1586
regressed Number  515  von  1586
regressed Number  516  von  1586
regressed Number  517  von  1586
regressed Number  518  von  1586
regressed Number  519  von  1586
regressed Number  520  von  1586
regressed Number  521  von  1586
regressed Number  522  von  1586
regressed Number  523  von  1586
regressed Number  524  von  1586
regressed Number  525  von  1586
regressed Number  526  von  1586
regressed Number  527  von  1586
regressed Number  528  von  1586
regressed Number  529  von  1586
regressed Number  530  von  1586
regressed Number  531  von  1586
regressed 

regressed Number  751  von  1586
regressed Number  752  von  1586
regressed Number  753  von  1586
regressed Number  754  von  1586
regressed Number  755  von  1586
regressed Number  756  von  1586
regressed Number  757  von  1586
regressed Number  758  von  1586
regressed Number  759  von  1586
regressed Number  760  von  1586
regressed Number  761  von  1586
regressed Number  762  von  1586
regressed Number  763  von  1586
regressed Number  764  von  1586
regressed Number  765  von  1586
regressed Number  766  von  1586
regressed Number  767  von  1586
regressed Number  768  von  1586
regressed Number  769  von  1586
regressed Number  770  von  1586
regressed Number  771  von  1586
regressed Number  772  von  1586
regressed Number  773  von  1586
regressed Number  774  von  1586
regressed Number  775  von  1586
regressed Number  776  von  1586
regressed Number  777  von  1586
regressed Number  778  von  1586
regressed Number  779  von  1586
regressed Number  780  von  1586
regressed 

regressed Number  1000  von  1586
regressed Number  1001  von  1586
regressed Number  1002  von  1586
regressed Number  1003  von  1586
regressed Number  1004  von  1586
regressed Number  1005  von  1586
regressed Number  1006  von  1586
regressed Number  1007  von  1586
regressed Number  1008  von  1586
regressed Number  1009  von  1586
regressed Number  1010  von  1586
regressed Number  1011  von  1586
regressed Number  1012  von  1586
regressed Number  1013  von  1586
regressed Number  1014  von  1586
regressed Number  1015  von  1586
regressed Number  1016  von  1586
regressed Number  1017  von  1586
regressed Number  1018  von  1586
regressed Number  1019  von  1586
regressed Number  1020  von  1586
regressed Number  1021  von  1586
regressed Number  1022  von  1586
regressed Number  1023  von  1586
regressed Number  1024  von  1586
regressed Number  1025  von  1586
regressed Number  1026  von  1586
regressed Number  1027  von  1586
regressed Number  1028  von  1586
regressed Numb

regressed Number  1241  von  1586
regressed Number  1242  von  1586
regressed Number  1243  von  1586
regressed Number  1244  von  1586
regressed Number  1245  von  1586
regressed Number  1246  von  1586
regressed Number  1247  von  1586
regressed Number  1248  von  1586
regressed Number  1249  von  1586
regressed Number  1250  von  1586
regressed Number  1251  von  1586
regressed Number  1252  von  1586
regressed Number  1253  von  1586
regressed Number  1254  von  1586
regressed Number  1255  von  1586
regressed Number  1256  von  1586
regressed Number  1257  von  1586
regressed Number  1258  von  1586
regressed Number  1259  von  1586
regressed Number  1260  von  1586
regressed Number  1261  von  1586
regressed Number  1262  von  1586
regressed Number  1263  von  1586
regressed Number  1264  von  1586
regressed Number  1265  von  1586
regressed Number  1266  von  1586
regressed Number  1267  von  1586
regressed Number  1268  von  1586
regressed Number  1269  von  1586
regressed Numb

In [14]:
df_regression_result.to_csv("regression_results.csv",sep=',') #Regrssionsergebnisse in csv Speichern

In [32]:
######Importiere Plotly Tools
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go 

"""
Graphische Darstellung von der Verteilung von Rsquared als Histogramm
"""


df_regression_result = pd.read_csv("regression_results.csv",sep=',') #Regressionsergebnisse wieder reinladen


#Histogrammobjekt erstellen
trace0 = go.Histogram(
    x= df_regression_result['RsquaredTest'],histnorm='probability',
    autobinx=False,
    xbins=dict(
        start=0,
        end=1,
        size=0.01
    ),
    cumulative=dict(enabled=True)
    
           

)

data = [trace0]
layout = go.Layout(
    title='RSquared',
    xaxis = dict(range=[0, 1])

)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cryto-regression')


In [33]:
####Einzelne Coins darstellen
###welcher Slug soll dargestellt Werden
SLUG_TO_DISPLAY = 'bitcoin'

##Regression durchfuehren
graphic_regression_data = cryptoregress(crypto_id= SLUG_TO_DISPLAY, feature_to_compare = 'close',crypto_id_feature_name="slug",
                 documentMode=False)


